this script is for converting the matsim travel times from aurore to an osm file which should be used as a "car" network in r5.

it is an alternative approach to properly matching the travel time onto the snman street graph.

In [12]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import geopandas as gpd
import shapely as shp

import snman
from snman.constants import *
from snman import osmnx_customized as oxc

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join(
    'C:',os.sep,'Users','lballo','polybox','Research',
    'SNMan','SNMan Shared','data_v2'
)
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)
paper_path = os.path.join(
    'C:',os.sep,'Users','lballo','polybox','Research',
    'E-Bike City Accessibility','EBC Accessibility Paper - Shared'
)
matsim_results_path = os.path.join(
    paper_path, 'MATSim results', '2024-07-12 Travel times before and after'
)

#CRS_internal = 29119    # for Boston
#CRS_internal = 32216    # for Chicago
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326
oxc.settings.useful_tags_way = OSM_TAGS

In [13]:
tt = pd.read_csv(
    os.path.join(matsim_results_path, 'before_bike100pct.csv')
)

tt

,LinkId,OSM_ID,FreeflowTTCar,FreeFlowTTBike,From_x,From_y,To_x,To_y,car_0:00,bike_0:00,...,bike_21:30,car_22:00,bike_22:00,car_22:30,bike_22:30,car_23:00,bike_23:00,car_23:30,bike_23:30,Unnamed: 104
0,3640,7749021,15.39,25.64,47.327088,8.381223,47.326734,8.380054,15.39,25.64,...,25.64,22.00,25.64,22.00,25.64,22.00,25.64,22.00,25.64,NaN
1,3638,7701086,22.48,22.48,47.499187,8.543563,47.499622,8.544628,22.48,22.48,...,22.48,28.00,22.48,22.48,22.48,22.48,22.48,22.48,22.48,NaN
2,3639,7701087,11.96,11.96,47.433009,8.668959,47.432571,8.669096,11.96,11.96,...,11.96,11.96,11.96,11.96,18.00,11.96,11.96,11.96,11.96,NaN
3,3630,7701079,40.03,40.03,47.490902,8.556413,47.492244,8.556424,40.03,40.03,...,40.03,46.00,40.03,40.03,40.03,40.03,40.03,46.00,40.03,NaN
4,3633,7642172,4.14,13.81,47.312737,8.913163,47.312457,8.913803,4.14,13.81,...,13.81,4.14,13.81,4.14,13.81,4.14,13.81,4.14,13.81,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143697,36256,7694151,30.34,30.34,47.536786,8.728047,47.536773,8.729726,30.34,30.34,...,36.00,30.34,30.34,30.34,30.34,30.34,30.34,30.34,30.34,NaN
143698,132450,7822078,19.23,19.23,47.395599,8.401946,47.396158,8.401275,19.23,19.23,...,19.23,26.00,19.23,26.00,19.23,19.23,19.23,25.50,19.23,NaN
143699,3629,7740686,46.18,46.18,47.349966,7.987588,47.351453,7.988677,46.18,46.18,...,46.18,53.00,46.18,46.18,46.18,53.00,46.18,53.00,46.18,NaN
143700,3628,7701078,21.15,21.15,47.492244,8.556424,47.492186,8.557590,21.15,21.15,...,21.15,21.15,21.15,28.00,28.00,21.15,28.00,21.15,21.15,NaN


In [14]:
import pyproj

transformer = pyproj.Transformer.from_crs(4326, 2056, always_xy=True)

tt['u'] = tt.apply(
    lambda row: shp.Point(
        *transformer.transform(row['From_y'], row['From_x'])
    ),
    axis=1
)
tt['v'] = tt.apply(
    lambda row: shp.Point(
        *transformer.transform(row['To_y'], row['To_x'])
    ),
    axis=1
)

tt['geometry'] = tt.apply(
    lambda row: shp.LineString([row['u'], row['v']]),
    axis=1
)

tt = gpd.GeoDataFrame(tt, crs=2056)

tt

,LinkId,OSM_ID,FreeflowTTCar,FreeFlowTTBike,From_x,From_y,To_x,To_y,car_0:00,bike_0:00,...,car_22:30,bike_22:30,car_23:00,bike_23:00,car_23:30,bike_23:30,Unnamed: 104,u,v,geometry
0,3640,7749021,15.39,25.64,47.327088,8.381223,47.326734,8.380054,15.39,25.64,...,22.00,25.64,22.00,25.64,22.00,25.64,NaN,POINT (2671251.845845772 1242229.8898536668),POINT (2671163.959879272 1242189.490882411),"LINESTRING (2671251.846 1242229.890, 2671163.9..."
1,3638,7701086,22.48,22.48,47.499187,8.543563,47.499622,8.544628,22.48,22.48,...,22.48,22.48,22.48,22.48,22.48,22.48,NaN,POINT (2683253.1287553683 1261522.6565044345),POINT (2683332.685601618 1261572.1175206336),"LINESTRING (2683253.129 1261522.657, 2683332.6..."
2,3639,7701087,11.96,11.96,47.433009,8.668959,47.432571,8.669096,11.96,11.96,...,11.96,18.00,11.96,11.96,11.96,11.96,NaN,POINT (2692815.897686968 1254306.1305690415),POINT (2692827.0037805545 1254257.5667423976),"LINESTRING (2692815.898 1254306.131, 2692827.0..."
3,3630,7701079,40.03,40.03,47.490902,8.556413,47.492244,8.556424,40.03,40.03,...,40.03,40.03,40.03,40.03,46.00,40.03,NaN,POINT (2684234.370337359 1260615.27174875),POINT (2684233.088362707 1260764.5139727283),"LINESTRING (2684234.370 1260615.272, 2684233.0..."
4,3633,7642172,4.14,13.81,47.312737,8.913163,47.312457,8.913803,4.14,13.81,...,4.14,13.81,4.14,13.81,4.14,13.81,NaN,POINT (2711487.843575352 1241253.987507178),POINT (2711536.8011244973 1241223.7886964267),"LINESTRING (2711487.844 1241253.988, 2711536.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143697,36256,7694151,30.34,30.34,47.536786,8.728047,47.536773,8.729726,30.34,30.34,...,30.34,30.34,30.34,30.34,30.34,30.34,NaN,POINT (2697083.4785140203 1265914.7378848481),POINT (2697209.8841212946 1265915.348891505),"LINESTRING (2697083.479 1265914.738, 2697209.8..."
143698,132450,7822078,19.23,19.23,47.395599,8.401946,47.396158,8.401275,19.23,19.23,...,26.00,19.23,19.23,19.23,25.50,19.23,NaN,POINT (2672724.7787943827 1249865.5624462888),POINT (2672673.3409902174 1249927.0092824404),"LINESTRING (2672724.779 1249865.562, 2672673.3..."
143699,3629,7740686,46.18,46.18,47.349966,7.987588,47.351453,7.988677,46.18,46.18,...,46.18,46.18,53.00,46.18,53.00,46.18,NaN,POINT (2641479.4981227415 1244490.8093179928),POINT (2641560.6089161052 1244656.6295373263),"LINESTRING (2641479.498 1244490.809, 2641560.6..."
143700,3628,7701078,21.15,21.15,47.492244,8.556424,47.492186,8.557590,21.15,21.15,...,28.00,28.00,21.15,28.00,21.15,21.15,NaN,POINT (2684233.088362707 1260764.5139727283),POINT (2684321.0478166896 1260759.2681966692),"LINESTRING (2684233.088 1260764.514, 2684321.0..."


In [15]:
nd = pd.concat([tt['u'], tt['v']]).reset_index().rename(columns={0: 'geometry'})
nd['osmid'] = pd.factorize(nd['geometry'])[0]
nd[['x', 'y']] = nd.apply(
    lambda row: (row['geometry'].x, row['geometry'].y),
    axis=1,
    result_type='expand'
)
nd.drop(columns=['index'], inplace=True)
nd.drop_duplicates(inplace=True)
nd.set_index('osmid', inplace=True)
nd = gpd.GeoDataFrame(nd, geometry='geometry', crs=2056)

nd

,geometry,x,y
osmid,,,
0,POINT (2671251.846 1242229.890),2.671252e+06,1.242230e+06
1,POINT (2683253.129 1261522.657),2.683253e+06,1.261523e+06
2,POINT (2692815.898 1254306.131),2.692816e+06,1.254306e+06
3,POINT (2684234.370 1260615.272),2.684234e+06,1.260615e+06
4,POINT (2711487.844 1241253.988),2.711488e+06,1.241254e+06
...,...,...,...
62684,POINT (2656358.412 1240891.713),2.656358e+06,1.240892e+06
62685,POINT (2678433.376 1240711.662),2.678433e+06,1.240712e+06
62686,POINT (2716835.861 1226991.701),2.716836e+06,1.226992e+06


In [16]:
tt = (pd.merge(tt, nd.reset_index()[['osmid', 'geometry']], how='left', left_on='u', right_on='geometry', suffixes=['', '_right'])
    .drop(columns=['u', 'geometry_right'])
    .rename(columns={'osmid': 'u'}))
tt = (pd.merge(tt, nd.reset_index()[['osmid', 'geometry']], how='left', left_on='v', right_on='geometry', suffixes=['', '_right'])
    .drop(columns=['v', 'geometry_right'])
    .rename(columns={'osmid': 'v'}))

tt.drop(columns=['From_x', 'From_y', 'To_x', 'To_y'], inplace=True)

tt['uv'] = tt.apply(lambda row: (row['u'], row['v']), axis=1)
tt.set_index('uv', inplace=True)

tt['length'] = tt.apply(lambda row: row.geometry.length, axis=1)
tt['travel_time_s'] = tt['car_7:00'].replace(0, 9999)
#tt['highway'] = 'primary'
tt['maxspeed'] = 40

tt

,LinkId,OSM_ID,FreeflowTTCar,FreeFlowTTBike,car_0:00,bike_0:00,car_0:30,bike_0:30,car_1:00,bike_1:00,...,bike_23:00,car_23:30,bike_23:30,Unnamed: 104,geometry,u,v,length,travel_time_s,maxspeed
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 19768)",3640,7749021,15.39,25.64,15.39,25.64,15.39,25.64,15.39,25.64,...,25.64,22.00,25.64,NaN,"LINESTRING (2671251.846 1242229.890, 2671163.9...",0,19768,96.726522,15.39,40
"(1, 14832)",3638,7701086,22.48,22.48,22.48,22.48,22.48,22.48,22.48,22.48,...,22.48,22.48,22.48,NaN,"LINESTRING (2683253.129 1261522.657, 2683332.6...",1,14832,93.678620,22.48,40
"(2, 20587)",3639,7701087,11.96,11.96,11.96,11.96,11.96,11.96,11.96,11.96,...,11.96,11.96,11.96,NaN,"LINESTRING (2692815.898 1254306.131, 2692827.0...",2,20587,49.817573,11.96,40
"(3, 62673)",3630,7701079,40.03,40.03,40.03,40.03,40.03,40.03,40.03,40.03,...,40.03,46.00,40.03,NaN,"LINESTRING (2684234.370 1260615.272, 2684233.0...",3,62673,149.247730,40.03,40
"(4, 16820)",3633,7642172,4.14,13.81,4.14,13.81,4.14,13.81,4.14,13.81,...,13.81,4.14,13.81,NaN,"LINESTRING (2711487.844 1241253.988, 2711536.8...",4,16820,57.522255,4.14,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(62681, 32587)",36256,7694151,30.34,30.34,30.34,30.34,30.34,30.34,30.34,30.34,...,30.34,30.34,30.34,NaN,"LINESTRING (2697083.479 1265914.738, 2697209.8...",62681,32587,126.407084,30.34,40
"(62599, 62382)",132450,7822078,19.23,19.23,19.23,19.23,19.23,19.23,19.23,19.23,...,19.23,25.50,19.23,NaN,"LINESTRING (2672724.779 1249865.562, 2672673.3...",62599,62382,80.134645,19.23,40
"(62680, 25)",3629,7740686,46.18,46.18,46.18,46.18,46.18,46.18,46.18,46.18,...,46.18,53.00,46.18,NaN,"LINESTRING (2641479.498 1244490.809, 2641560.6...",62680,25,184.594978,46.18,40


In [17]:
#to fill in the missing highway tags, we load the original osm export
osm_export = snman.io.import_geofile_to_gdf(
    os.path.join(matsim_results_path, 'before_oneway_links.gpkg')
)

osm_export['osm_id'] = osm_export['osm_id'].astype('int64')
tt = pd.merge(
    tt.reset_index(), osm_export[['osm_id', 'highway']],
    left_on='OSM_ID', right_on='osm_id', how='left'
)
tt.set_index('uv', inplace=True)
tt

,LinkId,OSM_ID,FreeflowTTCar,FreeFlowTTBike,car_0:00,bike_0:00,car_0:30,bike_0:30,car_1:00,bike_1:00,...,bike_23:30,Unnamed: 104,geometry,u,v,length,travel_time_s,maxspeed,osm_id,highway
uv,,,,,,,,,,,,,,,,,,,,,
"(0, 19768)",3640,7749021,15.39,25.64,15.39,25.64,15.39,25.64,15.39,25.64,...,25.64,NaN,"LINESTRING (2671251.846 1242229.890, 2671163.9...",0,19768,96.726522,15.39,40,7749021,tertiary
"(1, 14832)",3638,7701086,22.48,22.48,22.48,22.48,22.48,22.48,22.48,22.48,...,22.48,NaN,"LINESTRING (2683253.129 1261522.657, 2683332.6...",1,14832,93.678620,22.48,40,7701086,residential
"(2, 20587)",3639,7701087,11.96,11.96,11.96,11.96,11.96,11.96,11.96,11.96,...,11.96,NaN,"LINESTRING (2692815.898 1254306.131, 2692827.0...",2,20587,49.817573,11.96,40,7701087,residential
"(3, 62673)",3630,7701079,40.03,40.03,40.03,40.03,40.03,40.03,40.03,40.03,...,40.03,NaN,"LINESTRING (2684234.370 1260615.272, 2684233.0...",3,62673,149.247730,40.03,40,7701079,residential
"(4, 16820)",3633,7642172,4.14,13.81,4.14,13.81,4.14,13.81,4.14,13.81,...,13.81,NaN,"LINESTRING (2711487.844 1241253.988, 2711536.8...",4,16820,57.522255,4.14,40,7642172,living_street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(62681, 32587)",36256,7694151,30.34,30.34,30.34,30.34,30.34,30.34,30.34,30.34,...,30.34,NaN,"LINESTRING (2697083.479 1265914.738, 2697209.8...",62681,32587,126.407084,30.34,40,7694151,residential
"(62599, 62382)",132450,7822078,19.23,19.23,19.23,19.23,19.23,19.23,19.23,19.23,...,19.23,NaN,"LINESTRING (2672724.779 1249865.562, 2672673.3...",62599,62382,80.134645,19.23,40,7822078,residential
"(62680, 25)",3629,7740686,46.18,46.18,46.18,46.18,46.18,46.18,46.18,46.18,...,46.18,NaN,"LINESTRING (2641479.498 1244490.809, 2641560.6...",62680,25,184.594978,46.18,40,7740686,residential


In [18]:
min(tt['length'])

1.04974657298051

In [19]:
G = snman.street_graph.street_graph_from_gdf(nd, tt)

In [20]:
snman.io.export_osm_xml(
    G,
    os.path.join(outputs_path, 'tt_car_before.osm'),
    tags=['LinkId', 'Freespeed', 'maxspeed', 'travel_time_s', 'length', 'highway'],
    uv_tags=True,
    overwrite_highway='primary',
    dont_overwrite_highway=['path'],
    set_maxspeed_by_cost='travel_time_s',
    raw_graph=True,
    floor_maxspeed=1,
    ceil_maxspeed=120
)

snman.io.osm_to_pbf(os.path.join(outputs_path, 'tt_car_before.osm'))

In [21]:
if 0:
    snman.io.export_street_graph(
        G,
        os.path.join(outputs_path, 'tt_after_edges.gpkg'),
        os.path.join(outputs_path, 'tt_after_nodes.gpkg'),
        crs=CRS_for_export
    )